<a href="https://colab.research.google.com/github/xuanthin/RunColabAutomatic/blob/main/LinkedIn_Connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def connect_linkedin(browser, url, email, subject, message, file, min_delay):
    browser.get(url)
    time.sleep(random.uniform(min_delay, min_delay + 3))

    #Customize message
    try:
        panel_left = browser.find_element_by_xpath('//div[contains(@class, "pv-text-details__left-panel")]').text
        name = panel_left.split('\n')[0]
        for i in [' (He/Him)', ' (She/Her)']:
            name = name.replace(i, '')

    except:
        name = ''
        pass

    subject = subject.replace('@subject', name)
    message = message.replace('@name', name)

    #Connect stay in More
    element_list = browser.find_elements_by_xpath('//button[contains(@aria-label, "More actions")]')
    for elementID in element_list:
        try:
            elementID.click()
            break
        except:
            pass

    #Clicks Connect button
    element_list = browser.find_elements_by_xpath('//*[text()="Connect"]')
    for elementID in element_list:
        try:
            elementID.click()
            break
        except:
            pass

    try:
        browser.find_element_by_xpath('//button[contains(@aria-label, "Connect")]').click()
    except:
        pass

    time.sleep(2)

    try:
        browser.find_element_by_xpath('//button[contains(@aria-label, "Add a note")]').click()
    except:
        pass

    actions = ActionChains(browser)
    for part in message.split('\n'):
        actions.send_keys(part)
        actions.key_down(Keys.SHIFT).send_keys(Keys.ENTER).key_up(Keys.SHIFT)
        time.sleep(2)

    actions.perform()
    time.sleep(2)

    # Add email to send
    try:
        browser.find_element_by_name('email').send_keys(email)
    except:
        pass

    send_button = browser.find_element_by_class_name('ml1')

    clickable = False
    time_wait = time.time()
    while not clickable and time.time() < time_wait + implicit_time_default:
        try:
            cursor = send_button.value_of_css_property('cursor')
            if cursor != 'not-allowed':
                clickable = True
            break
        except:
            continue

    if clickable:
        send_button.click()

    time.sleep(2)
    ActionChains(browser).send_keys(Keys.ESCAPE).perform()

    return name


def connect_linkedin_via_email(browser, email_list, min_delay):
    browser.get('https://www.linkedin.com/mynetwork/import-contacts/iwe/')
    time.sleep(random.uniform(min_delay, min_delay + 3))
    browser.find_element_by_xpath('//textarea[contains(@name, "message")]').click()

    actions = ActionChains(browser)
    actions.send_keys(email_list)
    actions.perform()
    time.sleep(2)

    browser.find_element_by_xpath('//button[contains(@id, "send-iwe")]').click()


def check_connection(browser, username, password, target_profile_url):
    api = Linkedin(username, password)
    target_profile = api.get_profile(target_profile_url)
    connections = api.get_connections(username)

    if target_profile.id in [c['id'] for c in connections]:
        return True
    else:
        return False